In [1]:
import json
import codecs
import copy
import re
import textwrap
import os
import pandas as pd

import string
from collections import Counter
import pandas as pd
from nltk.corpus import words as nltk_words, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from unidecode import unidecode
import re
from IPython.display import HTML

import spacy
import enchant
from collections import OrderedDict
import contractions

import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import string

# Download required resources
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger_eng')

pd.set_option('display.max_columns', 500)

## Tip 1: Know when and how to filter your words and tokens

In [ ]:
# text = "The researchers were studying the behaviors of 10 wolves that had been running, hunted, and howling throughout the forested regions. Interestingly, they noticed that the wolves’ activities varied depending on the season, with increased aggression observed during mating periods. The observation included analyzing journals written by those documenting the wolves’ movements and strategies for surviving in harsh environments. Understanding these interactions helps in predicting future behavioral patterns and classifying different subspecies accordingly."
# text = "It's Monday, and the researchers were studying the behaviors of 10 wolves that had been running, hunted, and howling throughout the forested regions. Interestingly, they noticed that the wolves’ activities varied depending on the season, with increased aggression observed during mating periods. The observation included analyzing journals written by those documenting the wolves’ movements and strategies for surviving in harsh environments. Understanding these interactions helps in predicting future behavioral patterns and classifying different subspecies accordingly."
text = "It's Monday, and the researchers were studying the behaviors of 10 wolves that had been running, hunted, and howling throughout the forested regions. Interestingly, they noticed that the wolves’ activities varied depending on the season, with increased aggression observed during mating periods. The observation included analyzing journals written by those documenting the wolves’ movements and strategies for surviving in harsh environments. Understanding these interactions helps in predicting future behavioral patterns and classifying different subspecies accordingly."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
spacy_tokens_with_punct = [token.text for token in doc]
spacy_tokens_wout_punct = [token.text for token in doc if not token.is_punct and not token.is_space]
spacy_tokens_alpha = [token.text for token in doc if token.is_alpha]

print(len(spacy_tokens_with_punct))
print(len(spacy_tokens_wout_punct))
print(len(spacy_tokens_alpha))



87
76
74


In [43]:
# f_list must be strings (names of attributes)
f_list = [
    "is_alpha", "is_ascii", "is_bracket", "is_currency", "is_digit",
    "is_left_punct", "is_lower", "is_oov", "is_punct", "is_quote",
    "is_right_punct", "is_sent_end", "is_sent_start", "is_space",
    "is_stop", "is_title", "is_upper"
]

# Now correctly access each attribute with getattr
results = [[getattr(token, attr) for attr in f_list] for token in doc]

# Show result
# for attr_name, value in zip(f_list, results):
#     print(f"{attr_name}: {value}")

# results, 
pd.DataFrame(data=results, columns=f_list, index=doc)

# results

,is_alpha,is_ascii,is_bracket,is_currency,is_digit,is_left_punct,is_lower,is_oov,is_punct,is_quote,is_right_punct,is_sent_end,is_sent_start,is_space,is_stop,is_title,is_upper
It,True,True,False,False,False,False,False,True,False,False,False,False,True,False,True,True,False
's,False,True,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False
Monday,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False
",",False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False
and,True,True,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
classifying,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
different,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
subspecies,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
accordingly,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False


## Tip 2: Detecting english words with pyenchant

In [ ]:
import spacy
import enchant
import pandas as pd

text = "Yesterday, I booked a table at a new Italian restaurant called Trattoria. The focaccia and bruschetta were great, but the cannoli was too sweet."
# text = "Yesterday, I booked a table at a new Mexican restaurant called Los Cumpadres. The enchiladas and chilaquiles were pretty tasty, but too spicy."

# Load English dictionary
english_dict = enchant.Dict("en_US")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Tokenization and checking
tokens = [token.text for token in doc if not token.is_space and not token.is_punct]

# Check if each token is an English word
is_english = [english_dict.check(token) for token in tokens]

df = pd.DataFrame({
    "Token": tokens,
    "Is English?": is_english
})

df


,Token,Is English?
0,Yesterday,True
1,I,True
2,booked,True
3,a,True
4,table,True
5,at,True
6,a,True
7,new,True
8,Italian,True
9,restaurant,True


## Tip 3: Expand contractions using the contractions package

In [48]:
import contractions

# Example text with contractions
text = "I'm learning NLP and it's amazing!" # I can't believe how easy it's getting.

# Expand contractions
expanded_text = contractions.fix(text)

print(f"Original text: {text}")
print(f"Expanded text: {expanded_text}")


Original text: I'm learning NLP and it's amazing!
Expanded text: I am learning NLP and it is amazing!


In [ ]:
import spacy
import pandas as pd

# Text
text = "I'm learning NLP and it's amazing!"  # I can't believe how easy it's getting.

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Create the three lists
spacy_tokens_with_punct = [token.text for token in doc]
spacy_tokens_wout_punct = [token.text for token in doc if not token.is_punct]
spacy_tokens_alpha = [token.text for token in doc if token.is_alpha]

# Create the unique index based on all tokens (with punctuations)
tokens = spacy_tokens_with_punct

# Build the DataFrame
df = pd.DataFrame(index=tokens)

# Fill columns
df["with_punct"] = df.index.map(lambda x: 'X' if x in spacy_tokens_with_punct else '')
df["without_punct"] = df.index.map(lambda x: 'X' if x in spacy_tokens_wout_punct else '')
df["alpha_only"] = df.index.map(lambda x: 'X' if x in spacy_tokens_alpha else '')


print(len(spacy_tokens_with_punct))
print(len(spacy_tokens_wout_punct))
print(len(spacy_tokens_alpha))


# Display
df

## Now, add the english word column

9
8
6


,with_punct,without_punct,alpha_only
I,X,X,X
'm,X,X,
learning,X,X,X
NLP,X,X,X
and,X,X,X
it,X,X,X
's,X,X,
amazing,X,X,X
!,X,,


In [15]:
dir(doc[0])

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_dep',
 'has_extension',
 'has_head',
 'has_morph',
 'has_vector',
 'head',
 'i',
 'idx',
 'iob_strings',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang

In [12]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment'